In [10]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy
import seaborn as sns

from steps.prepare_data import load_split_processed_data, process_train_data, process_test_data
from utils.model import predict_booster, load_model, Metrics, predict_xgbm, predict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
process_train_data()
process_test_data()

2024-07-05 00:27:48 - Running pipeline: TRANSFORM_TRAIN (0:09:36)
2024-07-05 00:27:48 - 0. Running step: set_missings (0:09:36)


Processing train data...
-----------------


2024-07-05 00:27:51 - 1. Running step: reduce_mem_usage (0:09:38)
2024-07-05 00:27:51 - 2. Running step: merge_train_dpi_features (0:09:39)


Mem. usage decreased to 231.19 Mb (75.0% reduction)


2024-07-05 00:27:51 - 3. Running step: merge_train_fe_features (0:09:39)
2024-07-05 00:27:52 - 4. Running step: merge_train_fe_total_features (0:09:40)
2024-07-05 00:27:52 - 5. Running step: merge_train_bnum_features (0:09:40)
2024-07-05 00:27:53 - 6. Running step: remove_abon_id_step (0:09:41)
2024-07-05 00:27:53 - 7. Running step: feature_selection (0:09:41)
2024-07-05 00:27:53 - 9. Done (0:09:41)


...

Data saved
Processing test data...
-----------------


2024-07-05 00:27:57 - 1. Running step: reduce_mem_usage (0:09:45)
2024-07-05 00:27:57 - 2. Running step: merge_test_dpi_features (0:09:45)


Mem. usage decreased to 225.88 Mb (75.5% reduction)


2024-07-05 00:27:58 - 3. Running step: merge_test_bnum_features (0:09:46)
2024-07-05 00:27:58 - 4. Running step: merge_test_fe_features (0:09:46)
2024-07-05 00:27:58 - 5. Running step: merge_test_fe_total_features (0:09:46)
2024-07-05 00:27:59 - 7. Done (0:09:47)


-----------------

Shape: (150000, 1467)
Columns: Index(['Ama_rchrgmnt_sum_max_mnt1', 'content_clc_mea_mnt1',
       'content_cnt_max_mnt1', 'voice_out_short_part_max_mnt1',
       'voice_mts_in_nrest_part_std_mnt1', 'num_act_days_max_mnt1',
       'sms_roam_clc_min_mnt1', 'voice_in_cmpttrs_avg_durmin_mnt1',
       'com_num_part_mea_mnt1', 'pay_avg_mea_mnt1',
       ...
       'accum_oth_dur_td_total', 'data_3g_tv_cnt_td_total',
       'voice_in_td_cnt_min_total', 'abon_part_td_total',
       'com_num_part_std_total', 'sms_in_cnt_std_total',
       'pay_p2p_out_sum_td_total', 'pay_max_td_total',
       'voice_in_short_part_td_total', 'voice_out_cmpttrs_avg_dumin_total'],
      dtype='object', length=1467)
Data saved


In [1]:
train_data, test_data = load_split_processed_data()

model_name = "LightGbmV2_LightGbmV2_all_features_v1_with_oversampling_1_priority_v4_auc_score.pickle"

train_data_x = train_data.drop(columns=["target"])
train_data_y = train_data["target"]

test_data_x = test_data.drop(columns=["target"])
test_data_y = test_data["target"]

print("-----------Train----------")
predict_booster(model_name, train_data)

print("-----------Test----------")
predict_booster(model_name, test_data)

model_first, features_firsts = load_model(model_name)

NameError: name 'load_split_processed_data' is not defined

In [1]:
train_data, test_data = load_split_processed_data()

model_name = "LightGbmV2_LightGbmV2_all_features_v1_with_oversampling_1_priority_v4_auc_score.pickle"

train_data_x = train_data.drop(columns=["target"])
train_data_y = train_data["target"]

test_data_x = test_data.drop(columns=["target"])
test_data_y = test_data["target"]

print("-----------Train----------")
predict_booster(model_name, train_data)

print("-----------Test----------")
predict_booster(model_name, test_data)

model_second, features_second = load_model(model_name)

NameError: name 'load_split_processed_data' is not defined

In [7]:
dataframe = test_data

features = features_firsts

X = dataframe[features]
y_true = dataframe.target

y_pred_proba_first = model_first.predict(X, num_iteration=model_first.best_iteration)
y_pred_proba_second = model_second.predict(X, num_iteration=model_second.best_iteration)

y_pred_proba = 0.4 * y_pred_proba_first + 0.6 * y_pred_proba_second

threshold = 0.5
y_pred = (y_pred_proba >= threshold).astype(int)

Metrics().call(y_true, y_pred, y_pred_proba)

NameError: name 'model_second' is not defined